In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def parse_review(review_html):
    # 解析评分
    score_span = review_html.find("span", class_="rating-other-user-rating")
    score = f"{score_span.span.text}/{score_span.span.find_next_sibling('span').text}" if score_span else "No score"

    # 解析标题
    title = review_html.find("a", class_="title").text.strip()

    # 解析用户名和日期
    user_link = review_html.find("span", class_="display-name-link")
    user = user_link.a.text.strip() if user_link else "Anonymous"
    date = review_html.find("span", class_="review-date").text.strip()

    # 解析短评内容
    content = review_html.find("div", class_="text").text.strip()

    # 解析“helpful”计数
    helpful_text = review_html.find("div", class_="actions").text.strip()
    helpful = helpful_text.split(" ")[0] if "out of" in helpful_text else "0"

    return {
        "score": score,
        "title": title,
        "user": user,
        "date": date,
        "content": content,
        "helpful": helpful
    }

def scrape_imdb_reviews(url, max_reviews=5):
    # 设置 Chrome WebDriver
    driver = webdriver.Chrome()

    # 打开 URL
    driver.get(url)

    reviews = []

    while len(reviews) < max_reviews:
        # 使用 BeautifulSoup 解析页面内容
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 解析评论
        for review_container in soup.find_all("div", class_="review-container"):
            review = parse_review(review_container)
            reviews.append(review)

        # 检查是否已达到所需评论数量
        if len(reviews) >= max_reviews:
            break

        # 点击“加载更多”按钮
        load_more_button = driver.find_element_by_id("load-more-trigger")
        if load_more_button:
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)  # 等待页面加载

    driver.quit()
    return reviews

# 使用示例
url = "https://www.imdb.com/title/tt1517268/reviews"
all_reviews = scrape_imdb_reviews(url)
all_reviews

AttributeError: 'NoneType' object has no attribute 'text'

In [2]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib.parse import quote

browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)
KEYWORD = 'iPad'

def index_page(page):
    """
    抓取索引页
    :param page: 页码
    """
    print(' 正在爬取第 ', page, ' 页 ')
    try:
        url = 'https://s.taobao.com/search?q=' + quote(KEYWORD)
        browser.get(url)
        if page > 1:
            input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mainsrp-pager div.form> input')))
            submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager div.form> span.btn.J_Submit')))
            input.clear()
            input.send_keys(page)
            submit.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, '#mainsrp-pager li.item.active> span'), str(page)))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.m-itemlist .items .item')))
        get_products()
    except TimeoutException:
        index_page(page)

from pyquery import PyQuery as pq
def get_products():
    """提取商品数据"""
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        product = {'image': item.find('.pic .img').attr('data-src'),
            'price': item.find('.price').text(),
            'deal': item.find('.deal-cnt').text(),
            'title': item.find('.title').text(),
            'shop': item.find('.shop').text(),
            'location': item.find('.location').text()}
        print(product)
        save_to_mysql(product)
        
import mysql.connector
from mysql.connector import Error

MYSQL_HOST = 'localhost'
MYSQL_DB = 'taobao'
MYSQL_USER = 'root'  # Replace with your MySQL username
MYSQL_PASS = 'mySQLrucZC'  # Replace with your MySQL password

connection = mysql.connector.connect(
    host=MYSQL_HOST,
    database=MYSQL_DB,
    user=MYSQL_USER,
    password=MYSQL_PASS
)

def save_to_mysql(result):
    """
    Save to MySQL database
    :param result: The result dictionary to save
    """
    try:
        cursor = connection.cursor()
        placeholders = ', '.join(['%s'] * len(result))
        columns = ', '.join(result.keys())
        sql = "INSERT INTO {} ({}) VALUES ({})".format('products', columns, placeholders)
        cursor.execute(sql, list(result.values()))
        connection.commit()
        print('Stored to MySQL successfully')
    except Error as e:
        print(f'Stored to MySQL failed: {e}')
    finally:
        cursor.close()

# Don't forget to close the connection when you are done
# connection.close()

        
MAX_PAGE = 100
import time
def main():
    """遍历每一页"""
    for i in range(1, MAX_PAGE + 1):
        index_page(i)
        time.sleep(5)
main()

 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 
 正在爬取第  1  页 


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF7CF5382B2+55298]
	(No symbol) [0x00007FF7CF4A5E02]
	(No symbol) [0x00007FF7CF3605AB]
	(No symbol) [0x00007FF7CF340038]
	(No symbol) [0x00007FF7CF3C6BC7]
	(No symbol) [0x00007FF7CF3DA15F]
	(No symbol) [0x00007FF7CF3C1E83]
	(No symbol) [0x00007FF7CF39670A]
	(No symbol) [0x00007FF7CF397964]
	GetHandleVerifier [0x00007FF7CF8B0AAB+3694587]
	GetHandleVerifier [0x00007FF7CF90728E+4048862]
	GetHandleVerifier [0x00007FF7CF8FF173+4015811]
	GetHandleVerifier [0x00007FF7CF5D47D6+695590]
	(No symbol) [0x00007FF7CF4B0CE8]
	(No symbol) [0x00007FF7CF4ACF34]
	(No symbol) [0x00007FF7CF4AD062]
	(No symbol) [0x00007FF7CF49D3A3]
	BaseThreadInitThunk [0x00007FFE389A7344+20]
	RtlUserThreadStart [0x00007FFE391626B1+33]


In [12]:
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyquery import PyQuery as pq
import time


KEYWORD ='美食'


#把chrome设为selenium驱动的浏览器代理；
browser = webdriver.Chrome()  #登录使用手机扫码
wait = WebDriverWait(browser, 10)



#打开淘宝网页，进行扫码登陆
def search_page():
    print('正在搜索')
    try:
        browser.get('https://www.taobao.com')
        
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#q"))
        )
        
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#J_TSearchForm > div.search-button > button')))
        input.send_keys(KEYWORD)
        submit.click()
        print(1)
        total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"body[data-spam='1'] #root > div > div[data-spm-anchor-id] > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div.Content--contentInner--QVTcU0M")))
        print(3)
        get_products()
        print(2)
        return total.text
    except TimeoutException:
        return search_page()
#进行翻页处理；
def next_page(page_number):
    print('正在翻页',page_number)
    try:
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#mainsrp-pager > div > div > div > div.form > input"))
        )
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager > div > div > div > div.form > span.btn.J_Submit')))
        input.clear()
        input.send_keys(page_number)
        submit.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'#mainsrp-pager > div > div > div > ul > li.item.active > span'),str(page_number)))
        get_products()
    except TimeoutException:
        next_page(page_number)
#获取每一页的商品信息；
def get_products():
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-itemlist .items .item')))
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        product = {
            'price':item.find('.price').text(),
            'deal':item.find('.deal-cnt').text()[:-3],
            'title':item.find('.title').text(),
            'shop':item.find('.shop').text(),
            'location':item.find('.location').text()
        }
        print(product)
        save_to_mongo(product)

#储存到数据库中
import mysql.connector

# MySQL数据库连接配置
mysql_config = {
    'user': 'root',
    'password': 'mySQLrucZC',
    'host': 'localhost',
    'database': 'taobao'
}

# 保存数据到MySQL
def save_to_mysql(result):
    try:
        conn = mysql.connector.connect(**mysql_config)
        cursor = conn.cursor()
        add_product = ("INSERT INTO meishi "
                       "(price, deal, title, shop, location) "
                       "VALUES (%s, %s, %s, %s, %s)")
        data_product = (result['price'], result['deal'], result['title'], result['shop'], result['location'])
        cursor.execute(add_product, data_product)
        conn.commit()
        cursor.close()
        conn.close()
        print('存储到MySQL成功', result)
    except mysql.connector.Error as err:
        print('存储到MySQL失败', result, "Error: {}".format(err))


#进行递归循环；
def main():
    try:
        total = search_page()
        total = int(re.compile('(\d+)').search(total).group(1))
        for i in range(2,total+1):
            time.sleep(2)
            next_page(i)
    except Exception:
        print('出错了')
    finally:
        browser.close()

#启动爬虫
if __name__ == '__main__':
    main()

正在搜索
1
正在搜索
1
正在搜索
1


KeyboardInterrupt: 

In [16]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from time import sleep
import json
 
#获取登录状态
class Login:
    def __init__(self):  # 全部设置为私有属性，封装类
        self.driver = None
        self.__init_browser()  # 初始化浏览器
        self.__wait = WebDriverWait(self.driver, 180)  # 显示等待
        self.__main()  # 该方法也可以在外部调用
 
    def __init_browser(self) -> "初始化 浏览器":
        __options = ChromeOptions()
        __options.add_experimental_option('excludeSwitcher',
                                          ['enable-automation'])  # 设置开发者模式启动，该模式下webdriver属性为正常值
        __option = ChromeOptions()
        __option.add_argument(f'user-agent={UserAgent().random}')  # 设置请求头
        # __option.add_argument(
        #     r'--user-data-dir=C:\Users\35005\AppData\Local\Google\Chrome\User Data\Default')  # 加载自己的数据
        self.driver = Chrome( options=__options)  # 添加到浏览器中
        self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                    get: () => undefined
                })
            """
        })  # 避免淘宝滑块验证
 
    def __login(self) -> "登 录":
        self.driver.maximize_window()  # 最大化窗口
        self.driver.get("https://login.taobao.com/member/login.jhtml")  # 请求登录网页
        self.driver.find_element(By.XPATH, "//div[@id='login']/div[1]/i").click()  # 二维码登录
        self.__wait.until(expected_conditions.presence_of_element_located((By.ID, "q")))  # 等待二维码登录，知道搜索框出现
        # cookie = self.driver.execute_script("return document.cookie")  # 调用 JS 代码，获得cookie
        # self.driver.close()  # 关闭浏览器
        # return cookie
 
    def __main(self):
        self.__login()  # 调用接口
        sleep(5)  # 停止 5 秒钟
        # cookie = self.__login()
        # with open("../Config/cookie.pickle", "wb") as f:
        #     pickle.dump(cookie, f)  # 将数据存储到二进制文件中
 
if __name__ == '__main__':
    driver = Login()  # 直接调用 main 方法

InvalidArgumentException: Message: invalid argument: cannot parse capability: goog:chromeOptions
from invalid argument: unrecognized chrome option: excludeSwitcher
Stacktrace:
	GetHandleVerifier [0x00007FF70F6F82B2+55298]
	(No symbol) [0x00007FF70F665E02]
	(No symbol) [0x00007FF70F5205AB]
	(No symbol) [0x00007FF70F54300B]
	(No symbol) [0x00007FF70F54A798]
	(No symbol) [0x00007FF70F541682]
	(No symbol) [0x00007FF70F588AB4]
	(No symbol) [0x00007FF70F588694]
	(No symbol) [0x00007FF70F589EC0]
	(No symbol) [0x00007FF70F589CB0]
	(No symbol) [0x00007FF70F581E83]
	(No symbol) [0x00007FF70F55670A]
	(No symbol) [0x00007FF70F557964]
	GetHandleVerifier [0x00007FF70FA70AAB+3694587]
	GetHandleVerifier [0x00007FF70FAC728E+4048862]
	GetHandleVerifier [0x00007FF70FABF173+4015811]
	GetHandleVerifier [0x00007FF70F7947D6+695590]
	(No symbol) [0x00007FF70F670CE8]
	(No symbol) [0x00007FF70F66CF34]
	(No symbol) [0x00007FF70F66D062]
	(No symbol) [0x00007FF70F65D3A3]
	BaseThreadInitThunk [0x00007FFE389A7344+20

In [ ]:
数据来源：https://h5api.m.taobao.com/h5/mtop.relationrecommend.wirelessrecommend.recommend/2.0/

In [50]:
import requests

import requests

cookies = {
    'x5sec': '7b22617365727665723b32223a223331343166343065623537646534363765663661363631633464353034346430434e2f7739716f474549654c6a4b762b2f2f2f2f2f774561444451774d444d334d5449354f5459374d544343747532332f502f2f2f2f384251414d3d222c22733b32223a2261666561636634626231316662636263227d',
    '_samesite_flag_': 'true',
    'cookie2': '146c5b0b1b6fd2d528b158e61ca8e2c9',
    't': '7e2bab410332a32fb287860d05888fc7',
    '_tb_token_': '55e5333634b07',
    'xlly_s': '1',
    'cna': 'K40PHaTvtHUCAV+uRxu70EHC',
    'sgcookie': 'E100ZHcYstGsg2p4F4oVIXqBfGXK5fGWTIjqH%2F5kF9qICchJYtVDz9R0M87uwHfmhUkXhD0Y8avodTEYkIIACHC%2BOXvMXebtiZQRZzmbF1%2B%2BR0MSsM6uyaXvQdyRvZOlFucK',
    'unb': '4003712996',
    'uc1': 'cookie14=UoYelQxDRZ%2FQYQ%3D%3D&pas=0&existShop=false&cookie21=WqG3DMC9FxUx&cookie16=UtASsssmPlP%2Ff1IHDsDaPRu%2BPw%3D%3D&cookie15=Vq8l%2BKCLz3%2F65A%3D%3D',
    'uc3': 'vt3=F8dD3CN7Q7eTfu%2BsLqc%3D&lg2=Vq8l%2BKCLz3%2F65A%3D%3D&nk2=F5RAR4JQbN6grcc%3D&id2=VyyRoFpA%2F7jFIg%3D%3D',
    'csg': '74622d6a',
    'lgc': 'tb553216326',
    'cancelledSubSites': 'empty',
    'cookie17': 'VyyRoFpA%2F7jFIg%3D%3D',
    'dnk': 'tb553216326',
    'skt': 'ec62cb6737925444',
    'existShop': 'MTcwMDY0MTY2Mw%3D%3D',
    'uc4': 'nk4=0%40FY4L6woX6I%2Fl19ce9nhHFj54dpp20A%3D%3D&id4=0%40VXteVehui1K5nIMY0zes5i7VpRH2',
    'tracknick': 'tb553216326',
    '_cc_': 'W5iHLLyFfA%3D%3D',
    '_l_g_': 'Ug%3D%3D',
    'sg': '665',
    '_nk_': 'tb553216326',
    'cookie1': 'UtQOOsbSvp1NsHBurYxoRmt%2FIqIDxBW6B88eFFX%2FpkE%3D',
    '_m_h5_tk': '629bc6081fc13a6338043046bc7823bc_1700651024186',
    '_m_h5_tk_enc': '19654e6b4ce68324350d5901618c2f53',
    'tfstk': 'ddm6HB4k41f_WkUAbh9EO8yQYGrXUftP1twxExINHlETM2MoG5JDSluQGXhQ6GJMSrFjslUm7SPaGoMmNpRy43kiIoq9aQ-yZwHgmPAeL6-rIAqcCsprJ3SjYdHhUjjVZZKqnBsdAgS3dY9fe6fnARd0wANQKJmQCPN8JW6aegP7MPbfdzjbd7JBdZbDN_b4X',
    'l': 'fBNHcDWRPh86PkiAKOfwPurza77OSIRAguPzaNbMi9fPOACp5nhdW1eB4E89C3GVFs_2R3RxBjFXBeYBqC4qO_GJ2j-la_kmnmOk-Wf..',
    'isg': 'BHBwqR5s4S6d8r0gOpQ0xFVCQT7CuVQDy0VHKWrBMEufJRDPEsuDk3sbfS1FtQzb',
}

headers = {
    'authority': 'h5api.m.taobao.com',
    'accept': '*/*',
    'accept-language': 'zh-CN,zh;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'x5sec=7b22617365727665723b32223a223331343166343065623537646534363765663661363631633464353034346430434e2f7739716f474549654c6a4b762b2f2f2f2f2f774561444451774d444d334d5449354f5459374d544343747532332f502f2f2f2f384251414d3d222c22733b32223a2261666561636634626231316662636263227d; _samesite_flag_=true; cookie2=146c5b0b1b6fd2d528b158e61ca8e2c9; t=7e2bab410332a32fb287860d05888fc7; _tb_token_=55e5333634b07; xlly_s=1; cna=K40PHaTvtHUCAV+uRxu70EHC; sgcookie=E100ZHcYstGsg2p4F4oVIXqBfGXK5fGWTIjqH%2F5kF9qICchJYtVDz9R0M87uwHfmhUkXhD0Y8avodTEYkIIACHC%2BOXvMXebtiZQRZzmbF1%2B%2BR0MSsM6uyaXvQdyRvZOlFucK; unb=4003712996; uc1=cookie14=UoYelQxDRZ%2FQYQ%3D%3D&pas=0&existShop=false&cookie21=WqG3DMC9FxUx&cookie16=UtASsssmPlP%2Ff1IHDsDaPRu%2BPw%3D%3D&cookie15=Vq8l%2BKCLz3%2F65A%3D%3D; uc3=vt3=F8dD3CN7Q7eTfu%2BsLqc%3D&lg2=Vq8l%2BKCLz3%2F65A%3D%3D&nk2=F5RAR4JQbN6grcc%3D&id2=VyyRoFpA%2F7jFIg%3D%3D; csg=74622d6a; lgc=tb553216326; cancelledSubSites=empty; cookie17=VyyRoFpA%2F7jFIg%3D%3D; dnk=tb553216326; skt=ec62cb6737925444; existShop=MTcwMDY0MTY2Mw%3D%3D; uc4=nk4=0%40FY4L6woX6I%2Fl19ce9nhHFj54dpp20A%3D%3D&id4=0%40VXteVehui1K5nIMY0zes5i7VpRH2; tracknick=tb553216326; _cc_=W5iHLLyFfA%3D%3D; _l_g_=Ug%3D%3D; sg=665; _nk_=tb553216326; cookie1=UtQOOsbSvp1NsHBurYxoRmt%2FIqIDxBW6B88eFFX%2FpkE%3D; _m_h5_tk=629bc6081fc13a6338043046bc7823bc_1700651024186; _m_h5_tk_enc=19654e6b4ce68324350d5901618c2f53; tfstk=ddm6HB4k41f_WkUAbh9EO8yQYGrXUftP1twxExINHlETM2MoG5JDSluQGXhQ6GJMSrFjslUm7SPaGoMmNpRy43kiIoq9aQ-yZwHgmPAeL6-rIAqcCsprJ3SjYdHhUjjVZZKqnBsdAgS3dY9fe6fnARd0wANQKJmQCPN8JW6aegP7MPbfdzjbd7JBdZbDN_b4X; l=fBNHcDWRPh86PkiAKOfwPurza77OSIRAguPzaNbMi9fPOACp5nhdW1eB4E89C3GVFs_2R3RxBjFXBeYBqC4qO_GJ2j-la_kmnmOk-Wf..; isg=BHBwqR5s4S6d8r0gOpQ0xFVCQT7CuVQDy0VHKWrBMEufJRDPEsuDk3sbfS1FtQzb',
    'referer': 'https://s.taobao.com/',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

params = {
    'jsv': '2.6.2',
    'appKey': '12574478',
    't': '1700641663390',
    'sign': '44f7d40c2c59c555e561a44405c18c30',
    'api': 'mtop.relationrecommend.WirelessRecommend.recommend',
    'v': '2.0',
    'type': 'jsonp',
    'dataType': 'jsonp',
    'smToken': 'as',
    'sm': 'e',
    'callback': 'mtopjsonp3',
    'data': '{"appId":"34385","params":"{\\"device\\":\\"HMA-AL00\\",\\"isBeta\\":\\"false\\",\\"grayHair\\":\\"false\\",\\"from\\":\\"nt_history\\",\\"brand\\":\\"HUAWEI\\",\\"info\\":\\"wifi\\",\\"index\\":\\"4\\",\\"rainbow\\":\\"\\",\\"schemaType\\":\\"auction\\",\\"elderHome\\":\\"false\\",\\"isEnterSrpSearch\\":\\"true\\",\\"newSearch\\":\\"false\\",\\"network\\":\\"wifi\\",\\"subtype\\":\\"\\",\\"hasPreposeFilter\\":\\"false\\",\\"prepositionVersion\\":\\"v2\\",\\"client_os\\":\\"Android\\",\\"gpsEnabled\\":\\"false\\",\\"searchDoorFrom\\":\\"srp\\",\\"debug_rerankNewOpenCard\\":\\"false\\",\\"homePageVersion\\":\\"v7\\",\\"searchElderHomeOpen\\":\\"false\\",\\"search_action\\":\\"initiative\\",\\"sugg\\":\\"_4_1\\",\\"sversion\\":\\"13.6\\",\\"style\\":\\"list\\",\\"ttid\\":\\"600000@taobao_pc_10.7.0\\",\\"needTabs\\":\\"true\\",\\"areaCode\\":\\"CN\\",\\"vm\\":\\"nw\\",\\"countryNum\\":\\"156\\",\\"m\\":\\"pc\\",\\"page\\":1,\\"n\\":48,\\"q\\":\\"%E7%BE%BD%E7%BB%92%E6%9C%8D\\",\\"tab\\":\\"all\\",\\"pageSize\\":48,\\"totalPage\\":100,\\"totalResults\\":4800,\\"sourceS\\":\\"0\\",\\"sort\\":\\"_coefp\\",\\"bcoffset\\":\\"\\",\\"ntoffset\\":\\"\\",\\"filterTag\\":\\"\\",\\"service\\":\\"\\",\\"prop\\":\\"\\",\\"loc\\":\\"\\",\\"start_price\\":null,\\"end_price\\":null,\\"startPrice\\":null,\\"endPrice\\":null,\\"itemIds\\":null,\\"p4pIds\\":null}"}',
}

response = requests.get('https://h5api.m.taobao.com/h5/mtop.relationrecommend.wirelessrecommend.recommend/2.0/', params=params, cookies=cookies, headers=headers)

json_data = response.text[response.text.find('(')+1 : -1]

# 将字符串解析为JSON
data = json.loads(json_data)
data

{'api': 'mtop.relationrecommend.wirelessrecommend.recommend',
 'data': {'_cost': 526.382828,
  '_host': 'hippo.033124093237.na61',
  'version': 1.0,
  'iconStyle': {'202311tbxinfengchao': {'light': {'img': 'https://gw.alicdn.com/imgextra/i1/O1CN01JsLcWP1XegtVgdzUo_!!6000000002949-2-tps-109-42.png',
     'width': '109',
     'height': '42'},
    'dark': {'img': 'https://gw.alicdn.com/imgextra/i1/O1CN01JsLcWP1XegtVgdzUo_!!6000000002949-2-tps-109-42.png',
     'width': '109',
     'height': '42'}},
   'baoyounew': {'light': {'borderColor': '#FFA880',
     'bgColor': '#ffffff',
     'color': '#ff5000',
     'borderRadius': '2'},
    'dark': {'borderColor': '#FFA880',
     'bgColor': '#222222',
     'color': '#FF5000',
     'borderRadius': '2'}},
   'xinping': {'light': {'borderColor': '#FFA880',
     'bgColor': '#ffffff',
     'color': '#FF5000',
     'borderRadius': '2'},
    'dark': {'borderColor': '#A6725A',
     'bgColor': '#222222',
     'color': '#FF5000'}},
   'tianmaoxinfengchao2':

In [53]:
data['data']['itemsArray']

[{'clickUrl': 'https://click.simba.taobao.com/cc_im?p=%D3%F0%C8%DE%B7%FE&s=716551900&k=1153&e=gsoRDx2Q8qGDNAIJhhwm0%2FqJ%2BXWhKBkuwnWc5QL6aDVVSmrztFoF3hfQ%2BGX4o7qac%2FeFTuv7e5pQi9QiJeztQSGvoTUSCS9r6tNFmNWQb%2F9LOyYEUsW7%2B9vn9ssYHvDOvteY7FxgoYvwVGl473AwcfRVqemqpuR33XQso8B6BStL645TnY3Dqg49NjMxdeHyqlFx2Wp9SA6q06WA0dpyVeKhj9KIdfbGK9iJbpoTiH6M3mqhoEQN13Vw%2FzBtR0rb%2BqwygKzXOgCP3hKZi6GucvLIsRwMkoOQfX1x%2FiwUD3EiOx4nYp6Xb7AyS4W5kxyu1EzwzxAovlJ7cKpUaqo2pgukGBlvKGCQREVOwFEikVZIdVU2IEl88Oi1TkH3mdTrlV9jdCIN6d1nltCFPYsS8RcLlCq0bKQjsJCYSJnOkYmGaIVurhlOKLlP%2F%2FGlOvQqFYnGa4sKztYesSoDxeptb8UvyFgEW7U5qBnX%2FrU60waONvCQDxVuXuBf9StQsl8vr9cBRsxA5%2BaeA7JfkmyoPP%2BIFC02W%2BjE0iYckxfv3N2rq%2BlmsrnynY7u%2FxgGca5Ii8zLG3j0zrPMPx1SUFIrEyI9RgEh7TyldBrHpicp5nuZnmNAJ5GmTFE131S8VdPdsqdOdHZgoc%2FVPFsr%2BxcgcCji0kfz%2Bi2lLLaUEa6JoPANUS1kdDG14y1mQi7jzw65LIv13bfsRxs1sluYPdNURUOlHG6HFX%2F89cHNVmlBT7B1ayikMzl2XefbsxMM04JNce%2B%2B1vPog12f7dqk7BH75KisNUBZFInW6MVbuVttGGvdwWepEeI%2BDOkt7Qi0aFGwKH7gGDrv6bPigab4ha4cRBquVl

In [40]:
params = {
    'jsv': '2.6.2',
    'appKey': '12574478',
    't': '1700634267505',
    'sign': 'd2e1599479df916919280a87b73948d4',
    'api': 'mtop.relationrecommend.WirelessRecommend.recommend',
    'v': '2.0',
    'type': 'jsonp',
    'dataType': 'jsonp',
    'callback': 'mtopjsonp1',
    'data': '{"appId":"34385","params":"{\\"device\\":\\"HMA-AL00\\",\\"isBeta\\":\\"false\\",\\"grayHair\\":\\"false\\",\\"from\\":\\"nt_history\\",\\"brand\\":\\"HUAWEI\\",\\"info\\":\\"wifi\\",\\"index\\":\\"4\\",\\"rainbow\\":\\"\\",\\"schemaType\\":\\"auction\\",\\"elderHome\\":\\"false\\",\\"isEnterSrpSearch\\":\\"true\\",\\"newSearch\\":\\"false\\",\\"network\\":\\"wifi\\",\\"subtype\\":\\"\\",\\"hasPreposeFilter\\":\\"false\\",\\"prepositionVersion\\":\\"v2\\",\\"client_os\\":\\"Android\\",\\"gpsEnabled\\":\\"false\\",\\"searchDoorFrom\\":\\"srp\\",\\"debug_rerankNewOpenCard\\":\\"false\\",\\"homePageVersion\\":\\"v7\\",\\"searchElderHomeOpen\\":\\"false\\",\\"search_action\\":\\"initiative\\",\\"sugg\\":\\"_4_1\\",\\"sversion\\":\\"13.6\\",\\"style\\":\\"list\\",\\"ttid\\":\\"600000@taobao_pc_10.7.0\\",\\"needTabs\\":\\"true\\",\\"areaCode\\":\\"CN\\",\\"vm\\":\\"nw\\",\\"countryNum\\":\\"156\\",\\"m\\":\\"pc\\",\\"page\\":1,\\"n\\":48,\\"q\\":\\"%E7%BE%8E%E9%A3%9F\\",\\"tab\\":\\"all\\",\\"pageSize\\":48,\\"totalPage\\":100,\\"totalResults\\":4800,\\"sourceS\\":\\"0\\",\\"sort\\":\\"_coefp\\",\\"bcoffset\\":\\"\\",\\"ntoffset\\":\\"\\",\\"filterTag\\":\\"\\",\\"service\\":\\"\\",\\"prop\\":\\"\\",\\"loc\\":\\"\\",\\"start_price\\":null,\\"end_price\\":null,\\"startPrice\\":null,\\"endPrice\\":null,\\"itemIds\\":null,\\"p4pIds\\":null}"}',
}
import json

# 步骤 1: 获取data键对应的JSON字符串
data_str = params['data']

# 步骤 2: 解析JSON字符串为字典
data_dict = json.loads(data_str)

# 步骤 3: 解析嵌套的JSON字符串，并修改page值
params_dict = json.loads(data_dict['params'])
params_dict['page'] = 2  # 替换为您想要的新page值

# 步骤 4: 将修改后的内层字典转换回JSON字符串
updated_params_str = json.dumps(params_dict)
data_dict['params'] = updated_params_str

# 步骤 5: 将更新后的外层字典转换回JSON字符串，并更新params字典
params['data'] = json.dumps(data_dict)
params

{'jsv': '2.6.2',
 'appKey': '12574478',
 't': '1700634267505',
 'sign': 'd2e1599479df916919280a87b73948d4',
 'api': 'mtop.relationrecommend.WirelessRecommend.recommend',
 'v': '2.0',
 'type': 'jsonp',
 'dataType': 'jsonp',
 'callback': 'mtopjsonp1',
 'data': '{"appId": "34385", "params": "{\\"device\\": \\"HMA-AL00\\", \\"isBeta\\": \\"false\\", \\"grayHair\\": \\"false\\", \\"from\\": \\"nt_history\\", \\"brand\\": \\"HUAWEI\\", \\"info\\": \\"wifi\\", \\"index\\": \\"4\\", \\"rainbow\\": \\"\\", \\"schemaType\\": \\"auction\\", \\"elderHome\\": \\"false\\", \\"isEnterSrpSearch\\": \\"true\\", \\"newSearch\\": \\"false\\", \\"network\\": \\"wifi\\", \\"subtype\\": \\"\\", \\"hasPreposeFilter\\": \\"false\\", \\"prepositionVersion\\": \\"v2\\", \\"client_os\\": \\"Android\\", \\"gpsEnabled\\": \\"false\\", \\"searchDoorFrom\\": \\"srp\\", \\"debug_rerankNewOpenCard\\": \\"false\\", \\"homePageVersion\\": \\"v7\\", \\"searchElderHomeOpen\\": \\"false\\", \\"search_action\\": \\"initiati

In [33]:
import json
import csv


for item in data:
    title=item.get("title")
    priceWap=item.get("priceWap")
    realSales=item.get('realsales')
    procity=item.get('procity')
    nick=item.get('nick')
    shop_title=item.get('shopInfo').get("title")
    auctionURL=item.get('auctionURL')
    
    with open('taobao.csv',mode='a',newline='',encoding='utf-8') as f:
        csv_writer=csv.writer(f)
        csv_writer.writerow([title,priceWap,realSales,procity,nick,shop_title,auctionURL])


In [34]:
data

[{'clickUrl': 'https://click.simba.taobao.com/cc_im?p=%C3%C0%CA%B3&s=2114250112&k=1017&e=WhOEejEWFgECpLzg9%2BQ015dGU8dVqnhILGvbJO%2FFw0HCNRKCOjwjG9jr%2FBxZYq4Croq64hNJj0q37njhgun2RCr%2Bkj2cdLfIxupOCUyA8O%2BppIsnjKiBDXvyOk0xL3wmfp9T8HgHtzRfzL1MDGtKV9Lhg%2BtH%2BRm3%2BJ%2FVwvRuzt4zHhHSL7HKgh8VhbqpiDgmrG2uuluZEJcAhr88sU%2BQY6%2BKcRsv8NRLMytAqIIahJZ9uJGDAMend7uMIdO%2FtFFupnHyklogYB3CAUzHRpi4EuglpBkgOur%2BAV4d97fh6MAcQ9fPzDwyGtKpSK%2FRcYyQQ7qtOn0o1M8KtV8TMT8tOUo5u13IPUnW0iWjdO%2Bx7RSOVwtwHSKnf1WDhmQ%2B48IlbI3p7cRvXwEk48JC6LzUp2FLtOBjemvJIs6EP3C7d%2F%2Boytl6ujDTcAu3wD2D8YmjR2JWccTRo9lpR%2F0uExkUe8M6jnAi5oUJYCuQWLl%2BjpdQfvBer%2FjfgG9GEH2DstMH6rWSE6mA2%2BMkl1Nz4fEYN66Jin%2FjalC0bJUuxiGeeR0CySKVo1mHotYS9bdsdNm36nc74fKWuycQyKuFpF7fCWlyTam0MkEE%2B1lRWk34lLdfjQBgZdhN2FXWzkVGLHyVFh6TMe84L4RkrDcZQaNzo0hyK3%2BsUKIciX7v83Wm85j4o5ADURidLiPcNFINdRCbAYFFwuVDiPJ7%2BxFLSy0Zt7WT7Ji1orRrYY37uT9s4Lqd9O5QNUdhUwjqDXJ6K9Soy%2FoFMvZNI1niWkKtensnOr%2Bg8Ri0GKAZaDK5TpOZUxlWQGguup3f%2FSYuPv0SvBda3UPsWT3ffCvCRtbPKzgwl

In [ ]:
cookies = {
    'cna': 'K40PHaTvtHUCAV+uRxu70EHC',
    'lid': 'tb553216326',
    'xlly_s': '1',
    'dnk': 'tb553216326',
    'uc1': 'cookie15=W5iHLLyFOGW7aA%3D%3D&existShop=false&pas=0&cookie14=UoYelQ3O%2BB4ZmA%3D%3D&cookie21=Vq8l%2BKCLjA%2Bl&cookie16=W5iHLLyFPlMGbLDwA%2BdvAGZqLg%3D%3D',
    'uc3': 'nk2=F5RAR4JQbN6grcc%3D&lg2=Vq8l%2BKCLz3%2F65A%3D%3D&id2=VyyRoFpA%2F7jFIg%3D%3D&vt3=F8dD3CN6L0HiPDmHWZ4%3D',
    'tracknick': 'tb553216326',
    'uc4': 'id4=0%40VXteVehui1K5nIMY0zes54G9vycD&nk4=0%40FY4L6woX6I%2Fl19ce9nhHFj8dk2k%2BFQ%3D%3D',
    '_l_g_': 'Ug%3D%3D',
    'unb': '4003712996',
    'lgc': 'tb553216326',
    'cookie1': 'UtQOOsbSvp1NsHBurYxoRmt%2FIqIDxBW6B88eFFX%2FpkE%3D',
    'login': 'true',
    'cookie17': 'VyyRoFpA%2F7jFIg%3D%3D',
    'cookie2': '17729316b618de7ff72ea84a8aba87ee',
    '_nk_': 'tb553216326',
    'sgcookie': 'E100%2BuPhdF1BPXS3Mw0ReoVXJbm1Yi9GTNGn7G6pOH7OraukU4YKACGZtj%2FdU7HawzSAKLHsiGRa8MaO2%2BRJYMBuIYgcymFhohSM5%2F9hpGPTD%2BIEa4g6%2Bvw4YyqPxLzqGDUv',
    'cancelledSubSites': 'empty',
    't': '09d80db637348bc5fa5c71102d63d818',
    'sg': '665',
    'csg': '2254237c',
    '_tb_token_': '997e81303ead',
    '_m_h5_tk': '0c91ddc3a3d496fd872ab56cb8c17e67_1700728242490',
    '_m_h5_tk_enc': '6e3c7a0d937a4a3883426a42a0d1ca95',
    'isg': 'BOrqSD1Ke_th9vdX1LragyYIO1CMW2613QcNd3SjGz3lp4thXO_JxEWWNtO7YOZN',
    'l': 'fBg1BfwmPEl2rWBiBO5aPurza77OiIRb8fVzaNbMiIEGa1PNwKCRWNCTLM0JkdtxgT5AeetyMAhYGdh9Px438FkDBeYCKXIpBMJ68eOmrtHl.',
    'tfstk': 'dC6MXgf5Q1R1HOLGNPp_QcUCaBNLCV9XoZHvkKL4Te8CH1LY1-qD-gRTfZQAtEbF-FS9kl6ctNsy6Zjbfn8VuZYA6WeRfG9X3zQ0e8I_xKm5oXF-6KSu4Kz8y8EgO4vW31FX9XSObxEdvIkHqCzQHmQ8VRPW1USwSGIOLuRZ1GYhK9WejOphQ4lEa8vrhHrALjGX_HtHyKlZ3-f..',
}

headers = {
    'authority': 'h5api.m.tmall.com',
    'accept': '*/*',
    'accept-language': 'zh-CN,zh;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'cna=K40PHaTvtHUCAV+uRxu70EHC; lid=tb553216326; xlly_s=1; dnk=tb553216326; uc1=cookie15=W5iHLLyFOGW7aA%3D%3D&existShop=false&pas=0&cookie14=UoYelQ3O%2BB4ZmA%3D%3D&cookie21=Vq8l%2BKCLjA%2Bl&cookie16=W5iHLLyFPlMGbLDwA%2BdvAGZqLg%3D%3D; uc3=nk2=F5RAR4JQbN6grcc%3D&lg2=Vq8l%2BKCLz3%2F65A%3D%3D&id2=VyyRoFpA%2F7jFIg%3D%3D&vt3=F8dD3CN6L0HiPDmHWZ4%3D; tracknick=tb553216326; uc4=id4=0%40VXteVehui1K5nIMY0zes54G9vycD&nk4=0%40FY4L6woX6I%2Fl19ce9nhHFj8dk2k%2BFQ%3D%3D; _l_g_=Ug%3D%3D; unb=4003712996; lgc=tb553216326; cookie1=UtQOOsbSvp1NsHBurYxoRmt%2FIqIDxBW6B88eFFX%2FpkE%3D; login=true; cookie17=VyyRoFpA%2F7jFIg%3D%3D; cookie2=17729316b618de7ff72ea84a8aba87ee; _nk_=tb553216326; sgcookie=E100%2BuPhdF1BPXS3Mw0ReoVXJbm1Yi9GTNGn7G6pOH7OraukU4YKACGZtj%2FdU7HawzSAKLHsiGRa8MaO2%2BRJYMBuIYgcymFhohSM5%2F9hpGPTD%2BIEa4g6%2Bvw4YyqPxLzqGDUv; cancelledSubSites=empty; t=09d80db637348bc5fa5c71102d63d818; sg=665; csg=2254237c; _tb_token_=997e81303ead; _m_h5_tk=0c91ddc3a3d496fd872ab56cb8c17e67_1700728242490; _m_h5_tk_enc=6e3c7a0d937a4a3883426a42a0d1ca95; isg=BOrqSD1Ke_th9vdX1LragyYIO1CMW2613QcNd3SjGz3lp4thXO_JxEWWNtO7YOZN; l=fBg1BfwmPEl2rWBiBO5aPurza77OiIRb8fVzaNbMiIEGa1PNwKCRWNCTLM0JkdtxgT5AeetyMAhYGdh9Px438FkDBeYCKXIpBMJ68eOmrtHl.; tfstk=dC6MXgf5Q1R1HOLGNPp_QcUCaBNLCV9XoZHvkKL4Te8CH1LY1-qD-gRTfZQAtEbF-FS9kl6ctNsy6Zjbfn8VuZYA6WeRfG9X3zQ0e8I_xKm5oXF-6KSu4Kz8y8EgO4vW31FX9XSObxEdvIkHqCzQHmQ8VRPW1USwSGIOLuRZ1GYhK9WejOphQ4lEa8vrhHrALjGX_HtHyKlZ3-f..',
    'referer': 'https://detail.tmall.com/',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

params = {
    'jsv': '2.6.1',
    'appKey': '12574478',
    't': '1700721082023',
    'sign': '77bc8eadd15482848f451dc4ae585543',
    'api': 'mtop.taobao.pcdetail.data.get',
    'v': '1.0',
    'isSec': '0',
    'ecode': '0',
    'timeout': '10000',
    'ttid': '2022@taobao_litepc_9.17.0',
    'AntiFlood': 'true',
    'AntiCreep': 'true',
    'preventFallback': 'true',
    'type': 'jsonp',
    'dataType': 'jsonp',
    'callback': 'mtopjsonp1',
    'data': '{"id":"680658368469","detail_v":"3.3.2","exParams":"{\\"abbucket\\":\\"1\\",\\"id\\":\\"680658368469\\",\\"ns\\":\\"1\\",\\"spm\\":\\"a21n57.1.0.0.764f523c4wl8JW\\",\\"queryParams\\":\\"abbucket=1&id=680658368469&ns=1&spm=a21n57.1.0.0.764f523c4wl8JW\\",\\"domain\\":\\"https://detail.tmall.com\\",\\"path_name\\":\\"/item.htm\\"}"}',
}

response = requests.get('https://h5api.m.tmall.com/h5/mtop.taobao.pcdetail.data.get/1.0/', params=params, cookies=cookies, headers=headers)


In [59]:


elem = driver.find_element("id", "q")
human_login(elem, u"羽绒服")
time.sleep(0.2)
elem.send_keys(Keys.RETURN)


In [11]:
try:
    elem = driver.find_element("xpath", '//*[@id="root"]/div/div[2]/div[1]/div[1]/div[2]/div[3]/div/div[3]'
                                        '/a/div/div[3]/div[1]/a')
    elem.click()
except:
    try:
        elem = driver.find_element("xpath", '//*[@id="mainsrp-itemlist"]/div/div/div[1]/div[3]/div[2]/'
                                            'div[3]/div[1]/a/span[2]')
        elem.click()
    except:
        print("Xpath Fail")
        time.sleep(20)


In [ ]:
//*[@id="root"]/div/div[3]/div[1]/div[1]/div[2]/div[3]/div

#root > div > div:nth-child(3) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div
#root > div > div:nth-child(2) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div
#root > div > div:nth-child(3) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div

In [93]:
# 定位到包含所有商品的父div
from selenium.webdriver.common.by import By

try :
    parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(2) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
except:
    parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(3) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
# 获取所有子div（即所有商品的div）
product_divs = parent_div.find_elements(By.XPATH, "./div")

import csv

# 打开CSV文件
with open('downjacket.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["name", "price", "sales", "shop_name", "shop_link", "product_link", "image_link", "shipping_insurance", "free_shipping", "province", "city"])
    writer.writeheader()

# 遍历每个商品的div
i=0
for product_div in product_divs:
    # 提取商品名称
    # 注意：您需要根据实际页面结构调整选择器
    
    try:
        product_link = product_div.find_element(By.CSS_SELECTOR, 'a.Card--doubleCardWrapper--L2XFE73').get_attribute('href')

        # 商品名称
        product_name = product_div.find_element(By.CSS_SELECTOR, 'div.Title--title--jCOPvpf').text

        # 商品价格
        product_price = product_div.find_element(By.CSS_SELECTOR, 'span.Price--priceInt--ZlsSi_M').text

        # 商品销售量
        product_sales = product_div.find_element(By.CSS_SELECTOR, 'span.Price--realSales--FhTZc7U').text

        # 店铺名称
        shop_name = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').text

        # 网店链接
        shop_link = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').get_attribute('href')

        # 商品图片链接
        image_link = product_div.find_element(By.CSS_SELECTOR, 'img.MainPic--mainPic--rcLNaCv').get_attribute('src')
        
        location_elements = product_div.find_elements(By.CSS_SELECTOR, 'span.Price--procity--_7Vt3mX')
        province = location_elements[0].text if len(location_elements) > 0 else ''
        city = location_elements[1].text if len(location_elements) > 1 else ''


        # 是否包含运费险和包邮信息
        # 根据提供的HTML结构，这里假设如果存在相应的文本，则认为具备这些属性
        shipping_insurance = "赠运费险" in product_div.text
        free_shipping = "包邮" in product_div.text

        # 打印或保存数据
        with open('downjacket.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=["name", "price", "sales", "shop_name", "shop_link", "product_link", "image_link", "shipping_insurance", "free_shipping", "province", "city"])
            writer.writerow({
                    "name": product_name,
                    "price": product_price,
                    "sales": product_sales,
                    "shop_name": shop_name,
                    "shop_link": shop_link,
                    "product_link": product_link,
                    "image_link": image_link,
                    "shipping_insurance": shipping_insurance,
                    "free_shipping": free_shipping,
                    "province": province,
                    "city": city
                })
        print(i,':写入完成')
        i+=1
        #print(f"商品名称: {product_name}, 价格: {product_price}, 销售量: {product_sales}, 店铺名称: {shop_name}, 店铺链接: {shop_link}, 商品链接: {product_link}, 图片链接: {image_link}, 是否含运费险: {shipping_insurance}, 是否包邮: {free_shipping}")
    
    
    
    except Exception as e:
        print("发生错误:", e)
        print(product_div.text)
        break

0 :写入完成
1 :写入完成
2 :写入完成
3 :写入完成
4 :写入完成
5 :写入完成
6 :写入完成
7 :写入完成
8 :写入完成
9 :写入完成
10 :写入完成
11 :写入完成
12 :写入完成
13 :写入完成
14 :写入完成
15 :写入完成
16 :写入完成
17 :写入完成
18 :写入完成
19 :写入完成
20 :写入完成
21 :写入完成
22 :写入完成
23 :写入完成
24 :写入完成
25 :写入完成
26 :写入完成
27 :写入完成
28 :写入完成
29 :写入完成
30 :写入完成
31 :写入完成
32 :写入完成
33 :写入完成
34 :写入完成
35 :写入完成
36 :写入完成
37 :写入完成
38 :写入完成
39 :写入完成
40 :写入完成
41 :写入完成
42 :写入完成
43 :写入完成
44 :写入完成
45 :写入完成
46 :写入完成
47 :写入完成


## Have finished 10 pages 2023年11月23日时间

In [7]:
windows = driver.window_handles
driver.switch_to.window(windows[-1])
# 睡觉是好习惯，新的页面加载需要时间，有时候跳转新的
# 界面后立马操作也会出现元素不存在的情况
time.sleep(2)
//*[@id="root"]/div/div[3]/div[1]/div[1]/div[2]/div[4]/div/div/button[2]

In [100]:
from selenium.webdriver.support.ui import WebDriverWait


In [103]:
try:
        # 定位“下一页”按钮
    next_page_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div[1]/div[1]/div[2]/div[4]/div/div/button[2]')

    # 点击按钮
    next_page_button.click()


except NoSuchElementException:
    print("未找到'下一页'按钮")
except TimeoutException:
    print("等待新页面加载超时")
except Exception as e:
    print("发生翻页错误:", e)

In [107]:
last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(1):
    # 每次增加滚动的垂直位置
    for i in range(0, last_height, 100):  # 每次滚动100像素
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.1)  # 滚动间的小延时

    # 等待页面加载
    time.sleep(2)

    # 计算新的页面高度并与上次滚动后的页面高度进行比较
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # 如果高度没有变化，说明已到达页面底部
    last_height = new_height

In [110]:
total_height = driver.execute_script("return document.body.scrollHeight")

# 计算四分之三的位置
scroll_to = total_height * 0.7

# 每次滚动增加的高度
increment = 100  # 您可以根据需要调整这个值

# 当前滚动位置
current_scroll_position = 0

while current_scroll_position < scroll_to:
    # 增加滚动位置
    current_scroll_position += increment

    # 滚动到新位置
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

    # 短暂延时
    time.sleep(0.1)  # 每次滚动后的等待时间，可以根据需要调整

    # 防止滚动超出目标位置
    if current_scroll_position > scroll_to:
        driver.execute_script(f"window.scrollTo(0, {scroll_to});")
        break

In [104]:
with open('downjacket.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["name", "price", "sales", "shop_name", "shop_link", "product_link", "image_link", "shipping_insurance", "free_shipping", "province", "city"])
    writer.writeheader()

0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成


NameError: name 'NoSuchElementException' is not defined

In [111]:
for i in range(7,100):
    #滑动部分
    total_height = driver.execute_script("return document.body.scrollHeight")

    # 计算四分之三的位置
    scroll_to = total_height * 0.7

    # 每次滚动增加的高度
    increment = 100  # 您可以根据需要调整这个值

    # 当前滚动位置
    current_scroll_position = 0

    while current_scroll_position < scroll_to:
        # 增加滚动位置
        current_scroll_position += increment

        # 滚动到新位置
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

        # 短暂延时
        time.sleep(0.2)  # 每次滚动后的等待时间，可以根据需要调整

        # 防止滚动超出目标位置
        if current_scroll_position > scroll_to:
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            break
    time.sleep(3)
    #获取部分
    try :
        parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(2) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
    except:
        parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(3) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
    # 获取所有子div（即所有商品的div）
    product_divs = parent_div.find_elements(By.XPATH, "./div")

    # 打开CSV文件

    # 遍历每个商品的div
    i=0
    for product_div in product_divs:
        # 提取商品名称
        # 注意：您需要根据实际页面结构调整选择器

        try:
            product_link = product_div.find_element(By.CSS_SELECTOR, 'a.Card--doubleCardWrapper--L2XFE73').get_attribute('href')

            # 商品名称
            product_name = product_div.find_element(By.CSS_SELECTOR, 'div.Title--title--jCOPvpf').text

            # 商品价格
            product_price = product_div.find_element(By.CSS_SELECTOR, 'span.Price--priceInt--ZlsSi_M').text

            # 商品销售量
            product_sales = product_div.find_element(By.CSS_SELECTOR, 'span.Price--realSales--FhTZc7U').text

            # 店铺名称
            shop_name = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').text

            # 网店链接
            shop_link = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').get_attribute('href')

            # 商品图片链接
            image_link = product_div.find_element(By.CSS_SELECTOR, 'img.MainPic--mainPic--rcLNaCv').get_attribute('src')

            location_elements = product_div.find_elements(By.CSS_SELECTOR, 'span.Price--procity--_7Vt3mX')
            province = location_elements[0].text if len(location_elements) > 0 else ''
            city = location_elements[1].text if len(location_elements) > 1 else ''


            # 是否包含运费险和包邮信息
            # 根据提供的HTML结构，这里假设如果存在相应的文本，则认为具备这些属性
            shipping_insurance = "赠运费险" in product_div.text
            free_shipping = "包邮" in product_div.text

            # 打印或保存数据
            with open('downjacket.csv', 'a', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=["name", "price", "sales", "shop_name", "shop_link", "product_link", "image_link", "shipping_insurance", "free_shipping", "province", "city"])
                writer.writerow({
                        "name": product_name,
                        "price": product_price,
                        "sales": product_sales,
                        "shop_name": shop_name,
                        "shop_link": shop_link,
                        "product_link": product_link,
                        "image_link": image_link,
                        "shipping_insurance": shipping_insurance,
                        "free_shipping": free_shipping,
                        "province": province,
                        "city": city
                    })
            if i%10==0:print(i,':写入完成')
            i+=1
            #print(f"商品名称: {product_name}, 价格: {product_price}, 销售量: {product_sales}, 店铺名称: {shop_name}, 店铺链接: {shop_link}, 商品链接: {product_link}, 图片链接: {image_link}, 是否含运费险: {shipping_insurance}, 是否包邮: {free_shipping}")



        except Exception as e:
            print("发生写入或者读取错误:", e)
            print(product_div.text)
            break
            
            
    # 翻页部分
    try:
        # 定位“下一页”按钮
        next_page_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div[1]/div[1]/div[2]/div[4]/div/div/button[2]')

        # 点击按钮
        next_page_button.click()


    except NoSuchElementException:
        print("未找到'下一页'按钮")
    except TimeoutException:
        print("等待新页面加载超时")
    except Exception as e:
        print("发生翻页错误:", e)

0 :写入完成
10 :写入完成
20 :写入完成
30 :写入完成
40 :写入完成
发生写入或者读取错误: Message: no such element: Unable to locate element: {"method":"css selector","selector":"img.MainPic--mainPic--rcLNaCv"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E0A782B2+55298]
	(No symbol) [0x00007FF7E09E5E02]
	(No symbol) [0x00007FF7E08A05AB]
	(No symbol) [0x00007FF7E08E175C]
	(No symbol) [0x00007FF7E08E18DC]
	(No symbol) [0x00007FF7E08D819C]
	(No symbol) [0x00007FF7E09020EF]
	(No symbol) [0x00007FF7E08D80CF]
	(No symbol) [0x00007FF7E09022C0]
	(No symbol) [0x00007FF7E091AAA4]
	(No symbol) [0x00007FF7E0901E83]
	(No symbol) [0x00007FF7E08D670A]
	(No symbol) [0x00007FF7E08D7964]
	GetHandleVerifier [0x00007FF7E0DF0AAB+3694587]
	GetHandleVerifier [0x00007FF7E0E4728E+4048862]
	GetHandleVerifier [0x00007FF7E0E3F173+4015811]
	GetHandleVerifier [0x00007